<a href="https://colab.research.google.com/github/Mondin0/data-eng/blob/main/GabrielMondino_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción y almacenamiento de datos - Gabriel Mondino

En el presente trabajo realizo un trabajo de recolecciń de datos y guardado en un Data lakehouse, tomando datos de la [API de transporte oficial del Gobierno de CABA](https://api-transporte.buenosaires.gob.ar/)

En resumen, lo que hago es solicitar datos de las *estaciones de bicicletas [(ECO-BICI)](https://baecobici.com.ar/)*

Los requerimientos de la entrega parcial eran:

- Implementar técnicas de extracción de datos por medio del lenguaje de programación Python.
- Implementar técnicas de almacenamiento de datos, con el formato Delta lake.

La consigna de la primera entrega era:

Desarrollar un programa en Python que realice:
1. extracción de una API, como fuente de datos,
2. convierta los datos obtenidos como DataFrames de Pandas
3. y los guarde de forma cruda, sin transformaciones o con leves transformaciones, en formato Delta lake.

Por una cuestion de convención y para respetar buenas practicas, mantengo las credenciales en un archivo `credenciales.conf`.
que contiene la siguiente estructura:
```bash
[API]
client_id = ClientIdQueDaCABA
client_secret = ClientSecretQueDaCABA
url_station_information = https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation
url_station_status = https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus
```

La elección de esos dos endpoints en particular es que justo cumple con los dos requerimientos solicitados.

`Uno de los endpoints debe devolver datos temporales, que se actualicen periódicamente (mínimo
una vez al día), como por ejemplo: valores meteorológicos, cotizaciones de monedas o acciones de
compañías, variaciones de índices económicos, estadísticas deportivas, etc, El otro endpoint debe
ofrecer datos estáticos o metadatos, como por ejemplo campos o atributos...`

El endpoint `/ecobici/gbfs/stationInformation` trae un listado estático de todas las estaciones, sus capacidades y ubicaciones.


Por otro lado, el endpoint `/ecobici/gbfs/stationStatus` trae un listado del número de bicicletas y anclajes disponibles en cada estación y disponibilidad de estación(Datos temporales).


Uso extracción full debido a que no descubri un filtro dentro de la API de alguna fecha en particular. Lo obtenido hasta ahora estar


In [ ]:
!pip install requests
!pip install deltalake
!pip install pandas

In [ ]:
import configparser
import requests

def obtener_datos_api():

    """
    Función principal que se encarga de traer informacion de la api, enviando lo necesario segun la documentación de la misma.
    Gestiona posibles errores de conexión a la API y retorna información útil para el debuggeo.
    """

    # Leer las credenciales y URLs desde el archivo
    config = configparser.ConfigParser()
    config.read('credenciales.conf')

    # Obtener las credenciales y URLs
    client_id = config['API']['client_id']
    client_secret = config['API']['client_secret']
    url_station_information = config['API']['url_station_information']
    url_station_status = config['API']['url_station_status']

    # Parámetros de autenticación
    params = {
        "client_id": client_id,
        "client_secret": client_secret
    }

    # Realizar las solicitudes GET
    try:
        response_info = requests.get(url_station_information, params=params) #Endpoint estatico
        response_status = requests.get(url_station_status, params=params) #Endpoint dinamico

        # Verificar si las solicitudes fueron exitosas
        if response_info.status_code == 200 and response_status.status_code == 200:
            print("Solicitudes exitosas. Datos obtenidos")
            datos_info = response_info.json()
            datos_status = response_status.json()
            return datos_info, datos_status
        else:
            print("Error en la solicitud:")
            print(f"Status Code para {url_station_information}: {response_info.status_code}")
            print(f"Status Code para {url_station_status}: {response_status.status_code}")
            return None
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return None


In [ ]:
# Llamar a la función para obtener los datos en primera instancia
datos = obtener_datos_api()
if datos:
    datos_info, datos_status = datos
    print("Datos de información de estaciones:")
    print(datos_info)
    print("Datos de estado de estaciones:")
    print(datos_status)

#Paso 2: Conversión a DataFrame de PANDAS

Con PANDAS generamos dos dataframes: le llamaremos df_info al de informacion y df_status al estado en tiempo real de las estaciones de bicis. hacemos modificaciones pertinentes sobre filtrar valores nulos o NaN

In [ ]:
import pandas as pd
import json
import numpy as np

# Llamar a la función
datos = obtener_datos_api()
if datos:
    datos_info, datos_status = datos

    # Convertir los datos a DataFrames
    df_info = pd.json_normalize(datos_info['data']['stations'])
    df_status = pd.json_normalize(datos_status['data']['stations'])

    # Reemplazar valores nulos con NaN para tipos numéricos y None para tipos de objeto
    df_info = df_info.replace({np.nan: None})  # Reemplaza NaN con None para tipos de objeto
    df_status = df_status.replace({np.nan: None})
    df_status['traffic'] = df_status['traffic'].fillna('Unknown') # Reemplazar valores nulos en la columna traffic

    # Asegurarse de que las columnas numéricas tengan NaN en lugar de None
    for col in df_info.columns:
        if df_info[col].dtype.kind in 'bifc':  # Tipos numéricos
            df_info[col] = df_info[col].replace({None: np.nan})

    for col in df_status.columns:
        if df_status[col].dtype.kind in 'bifc':  # Tipos numéricos
            df_status[col] = df_status[col].replace({None: np.nan})

    # Convierto la columna 'last_reported' a datetime desde UNIX
    df_status['last_reported'] = pd.to_datetime(df_status['last_reported'], unit='s')

    # Creo columnas para año, mes, día y hora. Para poder particionar
    df_status['anio'] = df_status['last_reported'].dt.year
    df_status['mes'] = df_status['last_reported'].dt.month
    df_status['dia'] = df_status['last_reported'].dt.day
    df_status['hora'] = df_status['last_reported'].dt.strftime('%H:%M:%S')


    # TO DO. Limpiar columnas no utiles

Solicitudes exitosas. Datos obtenidos


In [127]:
df_info.head()


,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,is_charging_station,rental_methods,groups,obcn,short_name,nearby_distance,_ride_code_support,cross_street
0,2,002 - Retiro I,SMARTLITMAPFRAME,-34.592424,-58.37471,0.0,AV. Dr. José María Ramos Mejía 1300,11111,40,False,"[KEY, TRANSITCARD, PHONE]",[RETIRO],,,1000.0,True,None
1,3,003 - ADUANA,SMARTLITMAPFRAME,-34.612207,-58.369129,None,Av. Paseo Colón 380,C1063,28,False,"[KEY, TRANSITCARD, PHONE]",[MONSERRAT],None,None,1000.0,True,.
2,4,004 - Plaza Roma,SMARTLITMAPFRAME,-34.603008,-58.368856,0.0,Av. Corrientes 100,11111,20,False,"[KEY, TRANSITCARD, PHONE]",[SAN NICOLAS],,,1000.0,True,None
3,5,005 - Plaza Italia,SMARTLITMAPFRAME,-34.58055,-58.420954,0.0,Av. Sarmiento 2601,1111,42,False,"[KEY, TRANSITCARD, PHONE]",[PALERMO],,,1000.0,True,None
4,6,006 - Parque Lezama,SMARTLITMAPFRAME,-34.628526,-58.369758,0.0,"Avenida Martin Garcia, 295",1111,20,False,"[KEY, TRANSITCARD, PHONE]",[SAN TELMO],,,1000.0,True,None


In [ ]:
df_status.head()

,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,anio,mes,dia,hora
0,2,4,0,36,0,2025-03-14 17:11:37,False,IN_SERVICE,1,1,1,Unknown,4,0,2025,3,14,17:11:37
1,3,2,2,24,0,2025-03-14 17:11:30,False,IN_SERVICE,1,1,1,Unknown,2,0,2025,3,14,17:11:30
2,4,0,11,9,0,2025-03-14 17:13:37,False,IN_SERVICE,1,1,1,Unknown,0,0,2025,3,14,17:13:37
3,5,3,2,37,1,2025-03-14 17:11:54,False,IN_SERVICE,1,1,1,Unknown,3,0,2025,3,14,17:11:54
4,6,3,7,10,1,2025-03-14 17:13:30,False,IN_SERVICE,1,1,1,Unknown,3,0,2025,3,14,17:13:30


# ETAPA 3: ALMACENAMIENTO EN DELTA LAKE

Vamos a utilizar DeltaLake para el almacenamiento en DeltaTables

In [ ]:
from deltalake import DeltaTable, write_deltalake

# Especificar la ruta donde se guardarán los archivos Delta Lake
ruta_info = "Datahouse/info"
ruta_status = "Datahouse/status"

# Guardar los DataFrames en formato Delta Lake
write_deltalake(ruta_info, df_info, mode='overwrite')
write_deltalake(ruta_status, df_status, mode='overwrite')

# Creo 2 DeltaTable2 para interactuar con la tabla
dt_info = DeltaTable(ruta_info)
dt_status = DeltaTable(ruta_status)

# Verificar que las tablas se han creado correctamente
print(dt_info.version())
print(dt_status.version())

0
0


In [ ]:
# Leer la tabla Delta como un DataFrame de Pandas
dt_info = DeltaTable(ruta_info).to_pandas()
df_info.head()

,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic,num_bikes_available_types.mechanical,num_bikes_available_types.ebike
0,2,4,0,36,0,1741971574,False,IN_SERVICE,1,1,1,Unknown,4,0
1,3,4,2,22,0,1741971567,False,IN_SERVICE,1,1,1,Unknown,4,0
2,4,2,11,7,0,1741971513,False,IN_SERVICE,1,1,1,Unknown,2,0
3,5,3,2,37,1,1741971591,False,IN_SERVICE,1,1,1,Unknown,3,0
4,6,3,7,10,1,1741971505,False,IN_SERVICE,1,1,1,Unknown,3,0


In [ ]:
dt_status = DeltaTable(ruta_status).to_pandas()
df_status.head()

,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,anio,mes,dia,hora
0,2,4,0,36,0,2025-03-14 17:11:37,False,IN_SERVICE,1,1,1,Unknown,4,0,2025,3,14,17:11:37
1,3,2,2,24,0,2025-03-14 17:11:30,False,IN_SERVICE,1,1,1,Unknown,2,0,2025,3,14,17:11:30
2,4,0,11,9,0,2025-03-14 17:13:37,False,IN_SERVICE,1,1,1,Unknown,0,0,2025,3,14,17:13:37
3,5,3,2,37,1,2025-03-14 17:11:54,False,IN_SERVICE,1,1,1,Unknown,3,0,2025,3,14,17:11:54
4,6,3,7,10,1,2025-03-14 17:13:30,False,IN_SERVICE,1,1,1,Unknown,3,0,2025,3,14,17:13:30
